In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "llm-jp/llm-jp-3-13b"  # adjust to your LLM-jp variant
batch_sequences = [
    "こんにちは、お元気ですか？",
    "機械学習はどのように進化していますか？"
]
last_ffn_layer_name = "<last_ffn_layer_name>"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")
model.eval().to("cuda")

# Freeze all params
for p in model.parameters():
    p.requires_grad = False

# Enable gradients for FFN layer only
last_ffn = dict(model.named_modules())[last_ffn_layer_name]
for p in last_ffn.parameters():
    p.requires_grad = True

inputs = tokenizer(batch_sequences, padding=True, truncation=True, return_tensors="pt").to(model.device)
input_ids = inputs["input_ids"]

labels = input_ids.clone()
labels[:, :-1] = input_ids[:, 1:]
labels[:, -1] = -100

outputs = model(input_ids, labels=labels)
loss = outputs.loss
loss.backward()

total_grad = 0.0
param_count = 0
for p in last_ffn.parameters():
    if p.grad is not None:
        total_grad += p.grad.abs().mean().item()
        param_count += 1

avg_grad = total_grad / param_count if param_count else 0.0
print(f"Average FFN gradient magnitude (batch): {avg_grad:.6f}")
